In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle

from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
import multiprocessing as mp
import importlib
from sklearn.model_selection import train_test_split

In [3]:
#tail -n +2 train.csv | split -l 150000


In [ ]:
columns = ['acoustic_data','time_to_failure']
path = 'train.csv.zip'


BATCH_SIZE=64
TIMESTEPS=150000

hdf_key = 'hdf_key'
df_cols_to_index = columns # list of columns (labels) that should be indexed
store = pd.HDFStore("train.hdf5")

for chunk in pd.read_csv(path, float_precision='round_trip', header=0, compression='zip',
                         chunksize=BATCH_SIZE*TIMESTEPS, iterator=True):
    # don't index data columns in each iteration - we'll do it later ...
    store.append(hdf_key, chunk, data_columns=df_cols_to_index, index=False)
    # index data columns in HDFStore

store.create_table_index(hdf_key, columns=df_cols_to_index, optlevel=9, kind='full')
store.close()

In [ ]:
path = 'train.csv.zip'
#
columns = ['acoustic_data','time_to_failure']

BATCH_SIZE=64
TIMESTEPS=150000

def get_chunk(idx):
    chunks = pd.read_csv(path, float_precision='round_trip', header=0, compression='zip',
                         chunksize=BATCH_SIZE*TIMESTEPS, iterator=True)
    for i,chunk in enumerate(chunks):
        if idx == i:
            return chunk

In [ ]:
chunks = pd.read_csv(path, float_precision='round_trip', header=0, compression='zip',
                         chunksize=BATCH_SIZE*TIMESTEPS, iterator=True)
start_time = time.time()

for i,chunk in enumerate(chunks):
    start_time = time.time()
    print(len(chunk))
    end_time = time.time()
    print("Elapsed time was %g seconds" % (end_time - start_time))

In [ ]:
for i in range(0,100):
    df = pd.read_csv("train/xaa", float_precision='round_trip', header=0, iterator=True)
    #df[["acoustic_data"]] = StandardScaler().fit_transform(df[["acoustic_data"]])
    #print(df.head())
    print(df.get_chunk(15000).describe())

In [9]:
path = 'train.csv.zip'


def get_generated_batch(bigchunk):
    #print("idx",idx)
    bigchunk_data = np.array_split(bigchunk['acoustic_data'].values, BATCH_SIZE)
    bigchunk_target = np.array_split(bigchunk['time_to_failure'].values, BATCH_SIZE)
    batch = np.empty((BATCH_SIZE,TIMESTEPS,1),dtype=float)
    target = np.empty((BATCH_SIZE,1),dtype=float)
    for i in range(0,BATCH_SIZE):
        #print(df.head())
        scaled = StandardScaler().fit_transform(bigchunk_data[i].reshape(-1,1))
        #print(i, len(df))
        #print(df["acoustic_data"].values.reshape(-1,1).shape)
        batch[i] = scaled
        target[i] = bigchunk_target[i][-1]
        #print(batch.shape)

    return batch, target

In [64]:
VAL_X = np.empty((384,TIMESTEPS,1),dtype=float)
VAL_Y = np.empty((384,1),dtype=float)


def get_batch():

    while 1:
        big_chunks = pd.read_csv(path, float_precision='round_trip', header=0, compression='zip',
                         chunksize=BATCH_SIZE*TIMESTEPS, iterator=True)
        val_i = 0
        for idx, big_chunk in enumerate(big_chunks):
            bigchunk_data = np.array_split(big_chunk['acoustic_data'].values, BATCH_SIZE)
            bigchunk_target = np.array_split(big_chunk['time_to_failure'].values, BATCH_SIZE)
            batch = np.empty((BATCH_SIZE,TIMESTEPS,1),dtype=float)
            target = np.empty((BATCH_SIZE,1),dtype=float)
            for i in range(0,BATCH_SIZE):                    
                #print(df.head())
                scaled = StandardScaler().fit_transform(bigchunk_data[i].reshape(-1,1))
                #print(i, len(df))
                #print(df["acoustic_data"].values.reshape(-1,1).shape)
                if len(scaled) < 150000:
                    big_chunks = pd.read_csv(path, float_precision='round_trip', header=0, compression='zip',
                         chunksize=BATCH_SIZE*TIMESTEPS, iterator=True)
                    continue
                batch[i] = scaled
                target[i] = bigchunk_target[i][-1]
                #print(batch.shape)
            if idx % 10 == 0:
                for j,b in enumerate(batch):
                    if val_i < 384:
                        VAL_X[val_i] = b
                        VAL_Y[val_i] = target[j]
                        val_i+=1
                continue
            else:
                yield (batch, target)


In [62]:
for i,val in enumerate(VAL_X):
    print(i, val)

0 [[ 0.08662787]
 [-1.30464183]
 [-0.74813395]
 ...
 [-0.19162607]
 [ 0.64313575]
 [ 0.92138969]]
1 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
2 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
3 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
4 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
5 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
6 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
7 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
8 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
9 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
10 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
11 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
12 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
13 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
14 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
15 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
16 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
17 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
18 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
19 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
20 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
21 [[0.]
 [

In [23]:
for i in range(0,66):
    b = next(get_batch())
    print(len([x for x in b[0]]))
    


64
64
64
64
64


KeyboardInterrupt: 

In [ ]:
from keras.utils import Sequence

class MY_Generator(Sequence):

    def __init__(self):
        pass

    #This function computes the number of batches that this generator is supposed to produce. 
    #So, we divide the number of total samples by the batch_size and return that value.    
    def __len__(self):
        return int(629145481/(64*150000))

    #Here, given the batch numberidx you need to put together a list that consists of data 
    #batch and the ground-truth (GT). In this example, we read a batch images of size 
    #self.batch and return an array of form[image_batch, GT]
    def __getitem__(self, idx):
        bigchunk = get_chunk(idx)
        train,Y = get_generated_batch(bigchunk)
        #print("idx",idx)
        #print("LOLILOL")
        #print(train.shape, Y.shape)

        return (train,Y)

In [72]:
#train_data, val_data, y_train, y_val = train_test_split(training, targets, test_size=0.1, random_state=42)

from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,Conv1D,TimeDistributed,MaxPooling1D,Flatten#,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.layers import Bidirectional

BATCH_SIZE=64
TIMESTEPS=150000

dropout=0.2


my_model = Sequential()
my_model.add(
        Conv1D(filters=25, kernel_size=1000, activation='relu',
               strides=1000, input_shape=(TIMESTEPS,1))
)
             
my_model.add(MaxPooling1D(pool_size=4))
#my_model.add(Flatten())


my_model.add(LSTM(use_bias = True,unit_forget_bias=True,units = 8))

my_model.add(Dense(1))

for layer in my_model.layers:
    print(layer.output_shape)

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
#my_model.summary()




callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=0),
    #Reseter()
]

#my_training_batch_generator = MY_Generator()

#####


#for i in range(0,100):
 #   print("epoch",i)
  #  big_chunks = pd.read_csv('train.csv.zip', float_precision='round_trip', header=0, compression='zip',
   #                      chunksize=BATCH_SIZE*TIMESTEPS, iterator=True)
    #epoch_losses = []
    #for j,big_chunk in enumerate(big_chunks):
     #   if j > BATCH_SIZE:
      #      continue
       # print("batch",j)
       # train, targets = get_generated_batch(big_chunk)
       # losses = my_model.train_on_batch(train, targets)
       # epoch_losses.append(losses[0])
    #print("epoch loss: ", np.mean(epoch_losses))



history = my_model.fit_generator(generator=get_batch(),
                                          steps_per_epoch=64,
                                          epochs=10,
                                          validation_data=(VAL_X, VAL_Y),
                                          callbacks=callbacks,
                                          #shuffle=False,
                                          #verbose=1,
                                          #validation_data=my_validation_batch_generator,
                                          #validation_steps=(num_validation_samples // batch_size),
                                          #use_multiprocessing=False,
                                          #workers=8,
                                          #max_queue_size=32
                      )


import matplotlib.pyplot as plt


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

(None, 150, 25)
(None, 37, 25)
(None, 8)
(None, 1)
Epoch 1/10
64/64 [==============================] - 162s 3s/step - loss: 4.1154 - mean_absolute_error: 4.1154 - val_loss: 2.6233 - val_mean_absolute_error: 2.6233
Epoch 2/10
64/64 [==============================] - 160s 2s/step - loss: 3.3100 - mean_absolute_error: 3.3100 - val_loss: 2.5627 - val_mean_absolute_error: 2.5627
Epoch 3/10
64/64 [==============================] - 160s 2s/step - loss: 3.1170 - mean_absolute_error: 3.1170 - val_loss: 2.5706 - val_mean_absolute_error: 2.5706
Epoch 4/10
64/64 [==============================] - 159s 2s/step - loss: 2.9499 - mean_absolute_error: 2.9499 - val_loss: 2.5882 - val_mean_absolute_error: 2.5882
Epoch 5/10
 7/64 [==>...........................] - ETA: 2:05 - loss: 4.3657 - mean_absolute_error: 4.3657

KeyboardInterrupt: 

In [ ]:
TEST_SPLITS='test'
test_splits = [f for f in listdir(TEST_SPLITS) if isfile(join(TEST_SPLITS, f))]

In [ ]:
test_splits

In [ ]:
test_split_chunks = np.array_split(test_splits,mp.cpu_count())

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS, True]) \
           for chunk in test_split_chunks]
    pool.close()
    pool.join()

In [ ]:
ids = []
preds = []
i=0
for r in res:
    for df in r.get():
        if i % 100 == 0:
            print(i)
        #training[i] = df.loc[:,df.columns != 'time_to_failure']
        ids.append(df['seg_id'].unique()[0].split(".")[0])
        test_df = df.drop('seg_id', axis=1)
        preds.append(my_model.predict(test_df.values.reshape(1,-1,NUMBER_OF_FEATURES))[0][0])
        i+=1

In [ ]:
submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = preds

submission.to_csv('submission.csv', index=False)

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission.head()

In [ ]:
preds

In [ ]:
res[0].get()[0]

In [ ]:
path = 'test/%s' % (np.random.choice(test_splits))
#

df = pd.read_csv(path, float_precision='round_trip', header=[0])


df.describe()